## Inference YoloR on COTS dataset (PART 2) - as simple as possible to help people start with YoloR and develop object detection solutions on Kaggle

This notebook introduces YOLOR on Kaggle and TensorFlow - Help Protect the Great Barrier Reef competition. It shows how to make inference using custom YoloR object detection model (COTS dataset). It could be good starting point for build own custom model based on YoloR detector and make object detection. Full github repository you can find here - [YOLOR](https://github.com/WongKinYiu/yolor). First part - TRAINING is here: https://www.kaggle.com/remekkinas/yolor-p6-w6-one-more-yolo-on-kaggle-train.

Steps covered in this INFERENCE notebook:

* Install modules
* Run YoloR inference on test images
* Submit to competition

<div class="alert alert-warning">I found that there is no reference custom YoloR model training and inference notebook on Kaggle. Since we have such an opportunity this is my contribution to this competition. Feel free to use it and enjoy! I really appreciate if you upvote this notebook. Thank you!</div>

<div class="alert alert-success" role="alert">
I introduced YoloX in TensorFlow - Help Protect the Great Barrier Reef competition as well. You can find these notebooks here:      
    <ul>
        <li> <a href="https://www.kaggle.com/remekkinas/yolox-full-training-pipeline-for-cots-dataset">YoloX full training pipeline for COTS dataset</a></li>
        <li> <a href="https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507">YoloX detections submission made on COTS dataset</a></li>
    </ul>
    
</div>

## Used, ran, forked? Please vote. It is a GREAT gift for me and a sign of thanks.

<div align="center"><img width="640" src="https://github.com/WongKinYiu/yolor/raw/main/figure/unifued_network.png"/></div>

<div align="center"><img width="640" src="https://github.com/WongKinYiu/yolor/raw/main/figure/performance.png"/></div>

## 0. MODULES INSTALATION 

In [ ]:
%cp -r ../input/yolor-kaggle-hub/yolor-lib .
%cp -r ../input/yolor-kaggle-hub/cocoapi .

In [ ]:
%cd /kaggle/working/cocoapi/PythonAPI

!make
!make install
!python setup.py install

In [ ]:
import pycocotools

%cd /kaggle/working/yolor-lib

In [ ]:
import torch
import cv2

import pandas as pd
import numpy as np

from utils.torch_utils import select_device
from PIL import Image

# YoloR methods
from models.models import Darknet
from utils.datasets import letterbox
from utils.general import non_max_suppression, scale_coords

In [ ]:
%cd /kaggle/working/

## 1. YoloR PREDICTION

In [ ]:
def show_prediction(img, bboxes, scores):

    for box, score in zip(bboxes, scores):
        cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0,0,255), 2)
        cv2.putText(img, f'{score:.2f}', (int(box[0]), int(box[1]-3)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1, cv2.LINE_AA)
    
    img = img[:,:,::-1]
    img = Image.fromarray(img).resize((1280, 720))
    return img

def load_yolor_model(model_path, cfg_path, img_size, device):
    
    model = Darknet(cfg_path).cuda()
    model.load_state_dict(torch.load(model_path, map_location=device)['model'])
    model.to(device).eval()
    
    init = torch.zeros((1, 3, img_size, img_size), device=device)
    _ = model(init) if device.type != 'cpu' else None 
    return model

def predict_img(model, img_in, img_size, conf_thres, iou_thres, device):
    bboxes = []
    scores = []
    classes = []
    
    img = letterbox(img_in, new_shape=img_size, auto_size=64)[0]
    img = np.ascontiguousarray(img[:, :, ::-1].transpose(2, 0, 1))
    img = (torch.from_numpy(img).to(device).float() / 255.0).unsqueeze(0)
    
    pred = model(img)[0]
    
    if pred is not None:
        pred = non_max_suppression(pred, conf_thres=conf_thres, iou_thres=iou_thres, merge=False, classes=None, agnostic=False)

    for i, det in enumerate(pred):
        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img_in.shape).round()    
    
    det = det.cpu().detach().numpy()
    
    bboxes = det[:, :4] 
    scores = det[:, 4] 
    classes = det[:, 5]
    
    return bboxes, scores, classes

### A. MODEL PARAMETERS

In [ ]:
DATASET_PATH = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
cfg_path = '/kaggle/working/yolor-lib/cfg/yolor_p6.cfg'
model_path = '/kaggle/input/yolor-kaggle-hub/yr_002.pt'

CONF = 0.01
IOU = 0.4
img_size = 2432

### B. LOAD MODEL

In [ ]:
device = select_device()
model = load_yolor_model(model_path, cfg_path, img_size, device)

### C. TEST ON IMAGES

In [ ]:
dir = f'{DATASET_PATH}'
imgs = [dir + f for f in ('video_1/4775.jpg', 'video_0/9794.jpg', 'video_0/4502.jpg', 'video_0/9651.jpg', 'video_0/9700.jpg',  'video_0/9674.jpg','video_0/20.jpg', 'video_0/17.jpg',)]

for fileimg in imgs:
    img = cv2.imread(fileimg)
    bboxes, scores, classes = predict_img(model, img, img_size, conf_thres = CONF, iou_thres = IOU, device = device)
    display(show_prediction(img, bboxes, scores))

## 2. SUBMISSION

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
for (image_np, sample_prediction_df) in iter_test:
    
    bboxes, scores, _ = predict_img(model, image_np[:,:,::-1], img_size, conf_thres = CONF, iou_thres = IOU, device = device)
    
    predictions = []
    detects = []
    
    for i in range(len(bboxes)):
        box = bboxes[i]
        score = scores[i]
        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        detects.append([x_min, y_min, x_max, y_max, score])
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

### Glad you got to the end of this notebook. Isn't it easy? Good luck! Remember to vote - it's important to me and it motivates me to share more ...